In [1]:
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv("../../../Raw Data/diabetic_data.csv")

## Initial Exploration

In [3]:
df

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,?,Pediatrics-Endocrinology,41,0,1,0,0,0,250.83,?,?,1,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,?,?,59,0,18,0,0,0,276,250.01,255,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,?,?,11,5,13,2,0,1,648,250,V27,6,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,?,?,44,1,16,0,0,0,8,250.43,403,7,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,?,?,51,0,8,0,0,0,197,157,250,5,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101761,443847548,100162476,AfricanAmerican,Male,[70-80),?,1,3,7,3,MC,?,51,0,16,0,0,0,250.13,291,458,9,None,>8,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Down,No,No,No,No,No,Ch,Yes,>30
101762,443847782,74694222,AfricanAmerican,Female,[80-90),?,1,4,5,5,MC,?,33,3,18,0,0,1,560,276,787,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,Yes,NO
101763,443854148,41088789,Caucasian,Male,[70-80),?,1,1,7,1,MC,?,53,0,9,1,0,0,38,590,296,13,None,None,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Down,No,No,No,No,No,Ch,Yes,NO
101764,443857166,31693671,Caucasian,Female,[80-90),?,2,3,7,10,MC,Surgery-General,45,2,21,0,0,1,996,285,998,9,None,None,No,No,No,No,No,No,Steady,No,No,Steady,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,NO


### Distribution of values for each medication
- Drop medication columns where all individuals have the same treatment: examide, citoglipton

In [4]:
value_counts_dict = {}

for column in ['metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone']:
    value_counts_dict[column] = df[column].value_counts()

value_counts_df = pd.DataFrame(value_counts_dict)

value_counts_df


,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone
Down,575,45,11,1,194,NaN,560,564,NaN,118,87,3,5,NaN,NaN,NaN,NaN,12218,6,NaN,NaN,NaN,NaN
No,81778,100227,101063,101680,96575,101765.0,89080,91116,101743.0,94438,95401,101458,101728,101763.0,101727.0,101766.0,101766.0,47383,101060,101753.0,101765.0,101764.0,101765.0
Steady,18346,1384,668,79,4670,1.0,11356,9274,23.0,6976,6100,295,31,3.0,38.0,NaN,NaN,30849,692,13.0,1.0,2.0,1.0
Up,1067,110,24,6,327,NaN,770,812,NaN,234,178,10,2,NaN,1.0,NaN,NaN,11316,8,NaN,NaN,NaN,NaN


### Check if diabetesMed is just the aggregation of medications
- Yes. It does not contain new information. 
- But we can still keep it for interpretation purposes?

In [5]:
df_med = df[['metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone']]

In [6]:
## "diabetesmed column is just the aggregation of medications"
df_med['med_agg'] = df_med.apply(lambda row: 'No' if all(val == 'No' for val in row) else 'Yes', axis=1)

<ipython-input-6-c2bddf3e2226>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_med['med_agg'] = df_med.apply(lambda row: 'No' if all(val == 'No' for val in row) else 'Yes', axis=1)


In [7]:
df[df['diabetesMed'] != df_med['med_agg']]

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted


## Clean Data

#### Drop columns
- Medications where everyone has the same value: 'examide', 'citoglipton'
- Medications where three or fewer people have different values: 'glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone', 'acetohexamide', 'troglitazone'
- Columns where most values are missing: 'weight', 'payer_code', 'medical_specialty'

In [9]:
## Drop columns
df2 = df.drop(columns = ['weight', 'payer_code', 'medical_specialty', 'examide', 'citoglipton', 'glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone', 'acetohexamide', 'troglitazone'])

#### Group medications

In [15]:
## Determine the change in medication for the entire group of medication
def determine_med_change(row):
    values = row.values
    if all(value == 'No' for value in values): ## When patient does not use any of the medications
        return 'NoMed'
    elif 'Up' in values and 'Down' in values:
        return 'Mixed'
    elif 'Up' in values:
        return 'Up'
    elif 'Down' in values:
        return 'Down'
    elif 'Steady' in values:
        return 'Steady'
    else:
        return 'Error' 

## Group medications into broader groups
def group_medications(df):
   
    # Biguanides
    group1 = ['metformin']

    # Sulfonylureas
    group2 = ['glipizide', 'glyburide', 'glimepiride', 'tolbutamide', 'chlorpropamide', 'tolazamide']

    # Meglitinides
    group3 = ['repaglinide', 'nateglinide']

    # Thiazolidinediones (TZDs)
    group4 = ['pioglitazone', 'rosiglitazone']

    # Alpha-glucosidase Inhibitors
    group5 = ['acarbose', 'miglitol']

    # Insulin and Combination Products
    group6 = ['insulin', 'glyburide-metformin', 'glipizide-metformin']
    
    df_out = df
    all_groups = [group1, group2, group3, group4, group5, group6]
    for i in range(len(all_groups)):
        cols = all_groups[i]
        colname_med_count = 'med_count_{}'.format(i + 1)
        colname_med_change = 'med_change_{}'.format(i + 1)

        # Count the number of medications prescribed across the entire group
        df_out[colname_med_count] = df[cols].apply(lambda x: (x != 'No').sum(), axis=1)

        # Determine the change in medication for the entire group 
        df_out[colname_med_change] = df[cols].apply(determine_med_change, axis=1)
        
    df_out['med_change_agg'] = df_out[['med_change_1', 'med_change_2', 
                            'med_change_3', 'med_change_4', 
                            'med_change_5', 'med_change_6']].apply(lambda row: 'No' if all(((val == 'NoMed') or (val == "Steady")) for val in row) else 'Ch', axis=1)
    df_out['med_change_other'] = np.where(
    (df_out['change'] == "Ch") & (df_out['med_change_agg'] == "No"), 
    "Yes", 
    "No")
#    df_out = df_out.drop(columns = group1 + group2 + group3 + group4 + group5 + group6)
    
    return df_out

In [16]:
df3 = group_medications(df2)

In [17]:
## Check if change == 'No' only if all group changes are 'No', OR all group changes are 'Steady'
df_med_change = df3[['med_change_1', 'med_change_2', 
                            'med_change_3', 'med_change_4', 
                            'med_change_5', 'med_change_6']]

df_med_change['change_agg'] = df_med_change.apply(lambda row: 'No' if all(((val == 'NoMed') or (val == "Steady")) for val in row) else 'Ch', axis=1)

<ipython-input-17-919481d0fe70>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_med_change['change_agg'] = df_med_change.apply(lambda row: 'No' if all(((val == 'NoMed') or (val == "Steady")) for val in row) else 'Ch', axis=1)


In [18]:
## Mismatches: 19308 rows
df3[(df3['change'] != df_med_change['change_agg'])]

## Check that among the rows we classify as "Change", there are no mismatches with the data's classification
## i.e. whenever we see a change in the medication columns provided, the data also says there is a change
df3[(df3['change'] != df_med_change['change_agg']) & (df_med_change['change_agg']=="Ch")]

## This means that the data classifies "Change" more often than we do
## Conclusion: 'change' column includes medication that is not in this dataset

## Based on this, modify code in group_medications to add columns "med_change_other" -> whether medication NOT in the dataframe has changed

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,tolazamide,insulin,glyburide-metformin,glipizide-metformin,change,diabetesMed,readmitted,med_count_1,med_change_1,med_count_2,med_change_2,med_count_3,med_change_3,med_count_4,med_change_4,med_count_5,med_change_5,med_count_6,med_change_6,med_change_agg,med_change_other


In [19]:
## Observation: 'Mixed' is very rare, indicating that the groupings are pretty good?
df3[df3['med_change_6']=="Mixed"]

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,tolazamide,insulin,glyburide-metformin,glipizide-metformin,change,diabetesMed,readmitted,med_count_1,med_change_1,med_count_2,med_change_2,med_count_3,med_change_3,med_count_4,med_change_4,med_count_5,med_change_5,med_count_6,med_change_6,med_change_agg,med_change_other


#### Group discharge types

In [20]:
# Map discharge_disposition_id to the group number
def map_to_group(disposition_id):
    
    group1 = [1, 7]  # Minimal to Low Severity (Independent or Minimal Care)
    group2 = [6, 8, 2, 10, 30, 29]  # Moderate Severity (Home Care or Short-term Facility Transfer)
    group3 = [12, 16, 17, 9, 15]  # Moderately High Severity (Specialized Outpatient Services)
    group4 = [3, 4, 5, 24, 22, 23]  # High Severity (Specialized Inpatient Care)
    group5 = [27, 28]  # Very High Severity (Specialized, Long-term, or Intensive Care)
    group6 = [13, 14, 11, 19, 20, 21]  # Most Severe (Palliative Care, Hospice, or Expired)
    group7 = [18, 25, 26]  # Unknown or Undefined (NULL, Not Mapped, Unknown/Invalid)

    
    if disposition_id in group1:
        return 1  
    elif disposition_id in group2:
        return 2  
    elif disposition_id in group3:
        return 3 
    elif disposition_id in group4:
        return 4  
    elif disposition_id in group5:
        return 5  
    elif disposition_id in group6:
        return 6 
    elif disposition_id in group7:
        return 7
    else:
        return 'Missing'

def group_discharge(df):
    df_out = df.copy()
    df_out['discharge_group'] = df['discharge_disposition_id'].apply(map_to_group)
#    df_out = df_out.drop(columns=['discharge_disposition_id'])
    
    return df_out

In [21]:
df4 = group_discharge(df3)

In [22]:
df4.to_csv('../output/cleaned_data_AJ.csv')

### Check missingness: medications, discharge_disposition_id, 

#### Medications

In [18]:
med_names = ['metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
                'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
                'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
                'tolazamide', 'examide', 'citoglipton', 'insulin',
                'glyburide-metformin', 'glipizide-metformin',
                'glimepiride-pioglitazone', 'metformin-rosiglitazone',
                'metformin-pioglitazone']

unique_values = {col: df[col].unique() for col in med_names}
unique_values

{'metformin': array(['No', 'Steady', 'Up', 'Down'], dtype=object),
 'repaglinide': array(['No', 'Up', 'Steady', 'Down'], dtype=object),
 'nateglinide': array(['No', 'Steady', 'Down', 'Up'], dtype=object),
 'chlorpropamide': array(['No', 'Steady', 'Down', 'Up'], dtype=object),
 'glimepiride': array(['No', 'Steady', 'Down', 'Up'], dtype=object),
 'acetohexamide': array(['No', 'Steady'], dtype=object),
 'glipizide': array(['No', 'Steady', 'Up', 'Down'], dtype=object),
 'glyburide': array(['No', 'Steady', 'Up', 'Down'], dtype=object),
 'tolbutamide': array(['No', 'Steady'], dtype=object),
 'pioglitazone': array(['No', 'Steady', 'Up', 'Down'], dtype=object),
 'rosiglitazone': array(['No', 'Steady', 'Up', 'Down'], dtype=object),
 'acarbose': array(['No', 'Steady', 'Up', 'Down'], dtype=object),
 'miglitol': array(['No', 'Steady', 'Down', 'Up'], dtype=object),
 'troglitazone': array(['No', 'Steady'], dtype=object),
 'tolazamide': array(['No', 'Steady', 'Up'], dtype=object),
 'examide': array([

- No missingness in medication columns

#### Discharge Dispositions

In [53]:
missing_ids = [18, 25, 26]
df_missing = df[df['discharge_disposition_id'].isin(missing_ids)]
df_not_missing = df[~df['discharge_disposition_id'].isin(missing_ids)]

In [54]:
## 4.5% of missing values
df_missing.shape[0]/df.shape[0]

0.04598785448971169

In [52]:
# Columns to analyze
categorical_cols = ['readmitted', 'change', 'diabetesMed']
continuous_cols = ['num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient','number_diagnoses']

# Getting distributions
for col in categorical_cols:
    df_new = pd.DataFrame()
    df_new['missing'] = df_missing[col].value_counts(normalize=True)
    df_new['not missing'] = df_not_missing[col].value_counts(normalize=True)
    print('Column: ', col)
    print(df_new)
    
for col in continuous_cols:
    df_new = pd.DataFrame()
    df_new['missing'] = [df_missing[col].mean()]
    df_new['not missing'] = [df_not_missing[col].mean()]
    print('Column: ', col)
    print(df_new)

Column:  readmitted
      missing  not missing
NO   0.582479     0.537029
>30  0.299786     0.351668
<30  0.117735     0.111303
Column:  change
     missing  not missing
No  0.676496     0.531374
Ch  0.323504     0.468626
Column:  diabetesMed
      missing  not missing
Yes  0.651709     0.775735
No   0.348291     0.224265
Column:  num_lab_procedures
    missing  not missing
0  42.18953     43.13932
Column:  num_procedures
    missing  not missing
0  1.520085     1.331036
Column:  num_medications
     missing  not missing
0  15.824359    16.031364
Column:  number_outpatient
    missing  not missing
0  0.119017     0.381425
Column:  number_emergency
    missing  not missing
0  0.046795     0.205117
Column:  number_inpatient
    missing  not missing
0  0.587607     0.637878
Column:  number_diagnoses
    missing  not missing
0  6.769872     7.454072


- MOST IMPORTANT: The distribution for the target variable 'readmitted' is similar for missing vs non-missing discharge codes
- Change in medications and being prescribed diabetesMed is less prevalent among missing discharge code patients.
- The average number of lab procedures, medications, diagnoses and inpatient/outpatient visit is lower among patients with missing discharge codes
- OVERALL: Seems like less severe patients are more likely to have missing discharge code, but the relationship is not strong. Could just treat as missing at random?

#### A1Cresult

In [57]:
df['A1Cresult'].value_counts()

None    84748
>8       8216
Norm     4990
>7       3812
Name: A1Cresult, dtype: int64

- No missing values. None indicates test was not taken.